In [68]:
import setup_runs as sr
import os, os.path
import pandas as pd
import numpy as np
import pyDOE as pyd

#export files?
export_ed_files_q = True



In [146]:

##########################################
###                                    ###
###    GENERATE EXPERIMENTAL DESIGN    ###
###                                    ###
##########################################

print("Starting build of components for experimental design.")


#number of lhs samples
n_lhs = sr.dict_init["n_lhs"]
#set baseline strategy
strat_baseline = 0#[x for x in sr.dict_strat.keys() if sr.dict_strat[x]["strategy_id"] == 0][0]

###   SOME ATTRIBUTE TABLES
df_attribute_design_id = pd.read_csv(sr.fp_csv_attribute_design)
#reduce
df_attribute_design_id = df_attribute_design_id[df_attribute_design_id["include"] == 1]
#get time series id
df_attribute_time_series_id = pd.read_csv(sr.fp_csv_attribute_time_series)


print("Check: export_ed_files_q = " + str(export_ed_files_q))
print("Check: n_lhs = " + str(n_lhs))




#################################
#    MULTI SECTOR COMPONENTS    #
#################################

#read in uncertainty table for additional sectors
parameter_table_additional_sectors = pd.read_csv(sr.fp_csv_parameter_ranges)
#reduce
parameter_table_additional_sectors = parameter_table_additional_sectors[parameter_table_additional_sectors["type"].isin(["incertidumbre", "accion"])]
#add field
parameter_table_additional_sectors["variable_name_lower"] = [x.lower().replace(" ", "_") for x in list(parameter_table_additional_sectors["parameter"])]
#fill nas and set to integer
parameter_table_additional_sectors["parameter_constant_q"] = parameter_table_additional_sectors["parameter_constant_q"].fillna(0)
parameter_table_additional_sectors["parameter_constant_q"] = np.array(parameter_table_additional_sectors["parameter_constant_q"]).astype(int)
#initialize available groups
groups_norm = set([int(x) for x in parameter_table_additional_sectors["normalize_group"] if not np.isnan(x)])

###   NORMALIZATION GROUP IDS

group_id = parameter_table_additional_sectors[["type", "parameter", "normalize_group"]].drop_duplicates()
#build normalize group and lever group ids (for deltas)
norm_vec = []

if len(groups_norm) > 0:
    #starting point for new group
    ind_group = max(groups_norm) + 1
else:
    ind_group = 1

#loop over rows
for i in range(0, len(group_id)):
    #get current group
    group_cur = group_id["normalize_group"].iloc[i]
    #test for NaN
    if np.isnan(group_cur):
        norm_vec = norm_vec + [ind_group]
        #next iteration
        ind_group += 1
    else:
        norm_vec = norm_vec + [int(group_cur)]

#add to group_id data frame
group_id["norm_group_id"] = norm_vec
group_id = group_id.reset_index(drop = True)


###   LEVER GROUP IDS

#temporary data frame to build lever group id with
group_id_tmp = group_id[["type", "norm_group_id"]].drop_duplicates()
group_id_tmp = group_id_tmp[group_id_tmp["type"].isin(["Accion", "accion"])]
group_id_tmp["lever_group_id"] = range(1, len(group_id_tmp) + 1)
if "lever_group_id" not in group_id.columns:
    group_id = pd.merge(group_id, group_id_tmp[["norm_group_id", "lever_group_id"]], how = "left", left_on = ["norm_group_id"], right_on = ["norm_group_id"])
#reduce
group_id = group_id[["parameter", "type", "norm_group_id", "lever_group_id"]]
#replace nas
group_id = group_id.fillna({"lever_group_id": -1})
#integer
group_id["lever_group_id"] = [int(x) for x in list(group_id["lever_group_id"])]
#merge in
group_id  = pd.merge(group_id, parameter_table_additional_sectors[["parameter", "variable_name_lower"]], how = "left", left_on = ["parameter"], right_on = ["parameter"])
group_id = group_id.drop_duplicates()
group_id = group_id[["parameter", "type", "variable_name_lower", "norm_group_id", "lever_group_id"]]


###   MERGE BACK IN TO PTAS AND CLEAN NAMES/DATA

#set merge fields and merge
fields_merge = list(set(parameter_table_additional_sectors.columns).intersection(set(group_id.columns)))
parameter_table_additional_sectors = pd.merge(parameter_table_additional_sectors, group_id, how = "left", left_on = fields_merge, right_on = fields_merge)
#fields to extract
fields_ext = [x for x in parameter_table_additional_sectors if (x not in ["normalize_group"])]
#clean the data frame
parameter_table_additional_sectors = parameter_table_additional_sectors[fields_ext]
parameter_table_additional_sectors = parameter_table_additional_sectors.dropna(axis = 1)
#dictionary to rename
dict_ptas_rename = dict([[x, x.lower().replace(" ", "_")] for x in parameter_table_additional_sectors.columns])
#set type
parameter_table_additional_sectors = parameter_table_additional_sectors.rename(columns = dict_ptas_rename)

#parameters to index
fields_add_sec_all_vals = ["strategy_id", "type", "parameter", "sector", "norm_group_id", "lever_group_id"]
#get parameter years that are defined
param_years_add_sec = [float(x) for x in parameter_table_additional_sectors.columns if x.replace(".", "").isnumeric()]
param_years_add_sec = [int(x) for x in param_years_add_sec if (int(x) == x)]
print("\n\nparam_years_add_sec:")
print(param_years_add_sec)
print(parameter_table_additional_sectors.columns)
print("\n\n")
#initialize for all parameters
all_vals_add_sec = {
    "param_years": param_years_add_sec,
    "future_id": list(range(1, n_lhs + 1)),
    "design_id": list(df_attribute_design_id["design_id"]),
    "time_series_id": list(set(df_attribute_time_series_id["time_series_id"]) & set(parameter_table_additional_sectors["time_series_id"]))
}
#sort some
all_vals_add_sec["time_series_id"].sort()

#loop
for field in fields_add_sec_all_vals:
    #set the field name
    str_field = field.lower().replace(" ", "_")
    #
    if field in ["lever_group_id", "norm_group_id"]:
        set_field = set([x for x in parameter_table_additional_sectors[field] if x > 0])
    else:
        set_field = set(parameter_table_additional_sectors[field])
    #update the dictionary
    all_vals_add_sec.update({str_field: set_field})




###################################
#    GENERATE ATTRIBUTE TABLES    #
###################################

###   RUN ID
df_attribute_run_id_0 = [(x, y) for x in sr.df_strat["strategy_id"] for y in ([0] + all_vals_add_sec["future_id"])]
df_attribute_run_id = pd.DataFrame(df_attribute_run_id_0)
df_attribute_run_id = df_attribute_run_id.rename(columns = {0: "strategy_id", 1: "future_id"})
df_attribute_run_id["run_id"] = range(0, len(df_attribute_run_id))
df_attribute_run_id = df_attribute_run_id[["run_id", "strategy_id", "future_id"]]

if export_ed_files_q:
    #note/export
    print("Exporting run_id attribute to " + sr.fp_csv_attribute_runs)
    df_attribute_run_id.to_csv(sr.fp_csv_attribute_runs, index = None)

###   MASTER ID
df_attribute_master_id = [[x] + [y] + list(z) for x in all_vals_add_sec["design_id"] for y in all_vals_add_sec["time_series_id"] for z in df_attribute_run_id_0]
df_attribute_master_id = pd.DataFrame(df_attribute_master_id)
df_attribute_master_id = df_attribute_master_id.rename(columns = {0: "design_id", 1: "time_series_id", 2: "strategy_id", 3: "future_id"})
fields_df_attribute_master_id = list(df_attribute_master_id.columns)
df_attribute_master_id["master_id"] = list(range(0, len(df_attribute_master_id)))
df_attribute_master_id = df_attribute_master_id[["master_id"] + fields_df_attribute_master_id]
df_attribute_master_id = pd.merge(df_attribute_master_id, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"])
#fields to order by
fields_ord_dfm = ["master_id", "design_id", "time_series_id", "run_id", "strategy_id", "future_id"]
#reorder
df_attribute_master_id = df_attribute_master_id[fields_ord_dfm].sort_values(by = fields_ord_dfm)

if export_ed_files_q:
    #note/export
    print("Exporting master_id attribute to " + sr.fp_csv_attribute_master)
    df_attribute_master_id.to_csv(sr.fp_csv_attribute_master, index = None)




#############################
#    GENERATE LHS MATRIX    #
#############################

print("Generating LHS Matrix...")
print("")
#dimensions
p_add_sec = len(all_vals_add_sec["parameter"])
n_add_sec_levers = len(all_vals_add_sec["lever_group_id"])
#generate latin hypercube sample
matrix_lhs = pyd.lhs(p_add_sec + n_add_sec_levers, samples = n_lhs)
#vector for delineating sub-matrices
vec_submat_lhs = [0, p_add_sec, n_add_sec_levers]
#set vector of submatrix index
vec_submat_lhs_names = ["add_sec", "levers"]

##  set names for additional sectors
dict_names_add_sec = [x.lower().replace(" ", "_") for x in all_vals_add_sec["parameter"]]
dict_names_add_sec.sort()
dict_names_add_sec = [[x, dict_names_add_sec[x]] for x in range(0, len(dict_names_add_sec))]
dict_names_add_sec = dict(dict_names_add_sec)

##  set names for lever groups
dict_names_levers = ["lever_group_" + str(x) for x in list(all_vals_add_sec["lever_group_id"])]
dict_names_levers = [[x, dict_names_levers[x]] for x in range(0, len(dict_names_levers))]
dict_names_levers = dict(dict_names_levers)

#initialize the fields for data frames
dict_submat_lhs_fields = {
    "add_sec": dict_names_add_sec,
    "levers": dict_names_levers
}
#setup the output file paths
dict_submat_file_paths = {
    "add_sec": sr.fp_csv_lhs_table_multi_sector,
    "levers": sr.fp_csv_lhs_table_levers
}

#initialize
dict_submat_lhs = {}
#break off components
for i in range(0, len(vec_submat_lhs) - 1):
    #set the field name for the dictionary
    nm = vec_submat_lhs_names[i]
    #check
    if i > -1:
        #get the indeces
        p0 = sum(vec_submat_lhs[0:(i + 1)])
    else:
        p0 = 0
    #set upper limit
    p1 = sum(vec_submat_lhs[0:(i + 2)])
    #temporary dataframe
    df_tmp = pd.DataFrame(matrix_lhs[:, p0:p1], index = None)
    #add name field
    if dict_submat_lhs_fields[nm] != None:
        df_tmp = df_tmp.rename(columns = dict_submat_lhs_fields[nm])
        #set names
        nms = list(df_tmp.columns)
        #add run id
        df_tmp["future_id"] = all_vals_add_sec["future_id"]
        #reorder
        df_tmp = df_tmp[["future_id"] + nms]
    #update dictionary
    dict_submat_lhs.update({nm: df_tmp})

    #export raw lhs data
    if export_ed_files_q:
        #note/export
        print("Exporting LHS for " + nm + " to " + dict_submat_file_paths[nm])
        df_tmp.to_csv(dict_submat_file_paths[nm], index = None)

##  SET SOME NAMES
fields_ed_add_sec = dict_submat_lhs["add_sec"].columns
fields_ordered_parameters = [x for x in fields_ed_add_sec if x != "future_id"]
#get parameters that are not ramped—they are constant across all years
all_constant_params = set(parameter_table_additional_sectors[parameter_table_additional_sectors["parameter_constant_q"] == 1]["parameter"])
all_constant_params = list(all_constant_params)
all_constant_params.sort()
#get indices
indices_fop_all_constant_params = [fields_ordered_parameters.index(x) for x in all_constant_params]


#################################################################
###                                                           ###
###    GENERATE EXPERIMENTAL DESIGN FOR ADDITIONAL SECTORS    ###
###                                                           ###
#################################################################

###############################
#    GENERATE LEVER DELTAS    #
###############################

print("Starting generation of lever deltas.")
print("")
##  START BUY BUILDING LONG TABLE OF TRANSFORMED LHS SAMPLES

dsl = dict_submat_lhs["levers"].copy()
dsl = pd.wide_to_long(dsl, i = ["future_id"], j = "lever_group_id", stubnames = "lever_group_")
dsl = dsl.reset_index()
dsl = dsl.rename(columns = {"lever_group_": "lhs_val"})
#data frame out
df_ld_lhs_transformed = []
#loop over design id
for did in all_vals_add_sec["design_id"]:
    #get applicable data
    dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
    #initialize
    lhs_out = dsl.copy()
    #get key for to_dict
    key = list(dict_data["vary_lever_deltas"].keys())[0]
    #get range of values
    vec_vals = np.array(lhs_out["lhs_val"])
    #set header for Future 0
    df_fut_0 = pd.DataFrame([[did, 0, x, 1] for x in all_vals_add_sec["lever_group_id"]], columns = ["design_id", "future_id", "lever_group_id", "lhs_val"])
    #check on varying
    if dict_data["vary_lever_deltas"][key] == 1:
        #transform
        m = float(dict_data["linear_transform_ld_m"][key])
        b = float(dict_data["linear_transform_ld_b"][key])
        #thresholds
        thresh_min = float(dict_data["min_lever_deltas"][key])
        thresh_max = float(dict_data["max_lever_deltas"][key])
        #transformation
        def linear_transform(x):
            return max(min(m*x + b, thresh_max), thresh_min)
        #updated vals
        vec_vals = list(map(linear_transform, vec_vals))
    else:
        #set the deltas to 1
        vec_vals = [1.0 for x in range(len(vec_vals))]
    #update in data frames
    lhs_out["lhs_val"] = vec_vals
    lhs_out["design_id"] = [did for x in range(len(vec_vals))]
    #update
    lhs_out = pd.concat([df_fut_0, lhs_out[["design_id", "future_id", "lever_group_id", "lhs_val"]]])
    #add
    df_ld_lhs_transformed = df_ld_lhs_transformed + [lhs_out]
#convert to dataframee
df_ld_lhs_transformed = pd.concat(df_ld_lhs_transformed)


##  THEN, GO BY STRATEGY TO BUILD DELTAS

print("Starting build of deltas by time series/strategy...")
print("")
#set strategies to build deltas for
strat_lever_deltas = [x for x in all_vals_add_sec["strategy_id"] if x != strat_baseline]
#temporary reduction
ptas_ld = parameter_table_additional_sectors[parameter_table_additional_sectors["lever_group_id"] > 0]

#merge field for generate lever deltas
fields_merge_ld = ["variable_name_lower", "lever_group_id"]
#set extraction fields by type
extraction_fields_ld = fields_merge_ld + [str(x) for x in param_years_add_sec]

#dictionary of lever deltas by strategy
dict_ld = {}
#fields to extract and use in transpose
fields_ext_ld = fields_ordered_parameters
#initialize
df_ld_shaped = []
#loop over time series ids
for ts_id in all_vals_add_sec["time_series_id"]:
    #get baseline data frame
    df_base = ptas_ld[(ptas_ld["strategy_id"] == strat_baseline) & (ptas_ld["time_series_id"] == ts_id)].copy()
    #reduce
    df_base = df_base[extraction_fields_ld]

    #loop over strategies to generate lever deltas by included years
    for strat in strat_lever_deltas:
        #get strategy id
        strat_id = int(strat)#int(sr.dict_strat_ids[strat])
        #get sub data frame
        df_strat = ptas_ld[(ptas_ld["strategy_id"] == strat) & (ptas_ld["time_series_id"] == ts_id)].copy()
        df_strat = df_strat[extraction_fields_ld]
        #data fields
        fields_data = [x for x in extraction_fields_ld if (x not in fields_merge_ld)]
        #get column rename
        dict_rename_df_strat = dict([[str(x), str(strat) + "_" + str(x)] for x in fields_data])
        #update
        df_strat = df_strat.rename(columns = dict_rename_df_strat)
        #merge in
        df_strat = pd.merge(df_strat, df_base, how = "inner", left_on = fields_merge_ld, right_on = fields_merge_ld)
        #set new lever delta fields
        fields_ld = []
        #generate subtraction
        for fd in fields_data:
            field_0 = fd
            field_s = str(strat) + "_" + str(fd)
            field_ld = "ld_" + field_s
            fields_ld = fields_ld + [field_ld]
            #parse out and get difference
            df_strat_tmp = df_strat[[field_0, field_s]].diff(axis = 1)
            #update
            df_strat[field_ld] = df_strat_tmp[field_s]
        #reduce
        df_strat = df_strat[fields_merge_ld + fields_ld]
        #merge in
        df_strat = pd.merge(df_strat, df_ld_lhs_transformed, how = "outer", left_on = ["lever_group_id"], right_on = ["lever_group_id"])
        #build new data
        df_deltas_adj = (np.array(df_strat[fields_ld]).transpose() * np.array(df_strat["lhs_val"])).transpose()
        df_deltas_adj = pd.DataFrame(df_deltas_adj, columns = fields_ld)
        #re-initialize
        df_strat = df_strat[["design_id", "future_id"] + fields_merge_ld]
        df_strat = pd.concat([df_strat, df_deltas_adj], axis = 1, sort = False)
        #update
        dict_ld.update({strat: df_strat})

        # BUILD RESHAPED VALUE

        #loop over design/future
        for did in list(df_strat["design_id"].unique()):
            for fut in list(df_strat["future_id"].unique()):
                df_tmp = df_strat[(df_strat["design_id"] == did) & (df_strat["future_id"] == fut)].copy()
                df_tmp = df_tmp[["variable_name_lower"] + fields_ld]
                #fields to add
                fields_new = list(df_tmp["variable_name_lower"])
                #new data frame
                df_tmp = pd.DataFrame(np.array(df_tmp[fields_ld]).transpose(), columns = fields_new)
                #add year
                df_tmp["year"] = param_years_add_sec
                #add design id and future
                df_tmp["design_id"] = [did for x in range(len(df_tmp))]
                df_tmp["future_id"] = [fut for x in range(len(df_tmp))]
                df_tmp["strategy_id"] = [strat_id for x in range(len(df_tmp))]
                df_tmp["time_series_id"] = [ts_id for x in range(len(df_tmp))]
                #order
                df_tmp = df_tmp[["design_id", "time_series_id", "strategy_id", "future_id", "year"] + fields_new]
                #update
                df_ld_shaped = df_ld_shaped + [df_tmp]
            #notify of completed reshape
            print("Reshaping of LHS table complete for design_id: " + str(did) + ", time_series_id: " + str(ts_id) + ", strategy_id: " + str(strat))

if len(df_ld_shaped) > 0:
    #convert to wide frame
    df_ld_shaped = pd.concat(df_ld_shaped)
    #rename
    df_ld_shaped = df_ld_shaped.rename(columns = dict([[x, "ld_" + x] for x in fields_new]))


##  CREATE RAMP VECTORS FOR UNCERTAINTY

y_0 = int(sr.dict_init["add_sec_variation_start_year"]) - 1
y_1 = max(param_years_add_sec)
y_base = min(param_years_add_sec)
vec_ramp_unc = sr.build_linear_mix_vec((y_0, y_1), (y_base, y_1))
#sr.build_mix_vec(0, 0, 0, "linear")#np.array([max(min((y - y_0)/(y_1 - y_0), 1), 0) for y in param_years_add_sec])
vec_ramp_base = 1 - vec_ramp_unc


##  BUILD BASIC PERCENTAGE CHANGE MATRIX


#for ts_id in all_vals_add_sec["time_series_id"]:
df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & (parameter_table_additional_sectors["time_series_id"] == 0)].copy()
df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
df_ed_add_sec = dict_submat_lhs["add_sec"]
#initialize array (in order of ext fields)
array_ed_add_sec = np.array(df_ed_add_sec[fields_ordered_parameters])
#order output array
df_max_min = pd.merge(pd.DataFrame(fields_ordered_parameters, columns = ["variable_name_lower"]), df_ed_baseline[["variable_name_lower", "min_2050", "max_2050"]], how = "left", left_on = ["variable_name_lower"], right_on = ["variable_name_lower"])
#generate transformed values of uncertainty
array_ed_add_sec_trans = array_ed_add_sec * np.array(df_max_min["max_2050"]) + (1 - array_ed_add_sec) * np.array(df_max_min["min_2050"])
#add in future 0 (0 change = 100%)
array_ed_add_sec_trans = np.concatenate([np.ones((1, array_ed_add_sec_trans.shape[1])), array_ed_add_sec_trans])
#set data frame of baseline percentage changes
df_perc_change_unc = []
#build outcome matrix
for i in range(len(param_years_add_sec)):
    y = param_years_add_sec[i]
    frac_base = vec_ramp_base[i]
    frac_unc = vec_ramp_unc[i]
    #build weighted array
    array_tmp = np.ones(array_ed_add_sec_trans.shape)*(frac_base) + array_ed_add_sec_trans*(frac_unc)
    #NOTE: FOR CONSTANT PARAMS, GET RID OF "RAMP"
    array_tmp[:, indices_fop_all_constant_params] = array_ed_add_sec_trans[:, indices_fop_all_constant_params].copy()
    #convert to data frame
    df_tmp = pd.DataFrame(array_tmp, columns = fields_ordered_parameters)
    #add year
    df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
    df_tmp["future_id"] = [0] + list(df_ed_add_sec["future_id"])
    #df_tmp["time_series_id"] = [ts_id for x in range(len(df_tmp))]
    #organize
    df_tmp = df_tmp[["future_id", "year"] + fields_ordered_parameters]
    #update
    df_perc_change_unc = df_perc_change_unc + [df_tmp]
#build master
df_perc_change_unc = pd.concat(df_perc_change_unc)
df_perc_change_unc = df_perc_change_unc.sort_values(by = ["future_id", "year"])


Starting build of components for experimental design.
Check: export_ed_files_q = True
Check: n_lhs = 10


param_years_add_sec:
[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050]
Index(['sector', 'time_series_id', 'strategy_id', 'type', 'parameter',
       'parameter_constant_q', 'min_2050', 'max_2050', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026',
       '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2035',
       '2036', '2037', '2038', '2039', '2040', '2041', '2042', '2043', '2044',
       '2045', '2046', '2047', '2048', '2049', '2050', 'variable_name_lower',
       'norm_group_id', 'lever_group_id'],
      dtype='object')



Exporting run_id attribute to /Users/jsyme/Documents/Projects/FY20/SWCHE076-1000/git/MultiSector_LTS_Chile/experimental_des

In [147]:
df_perc_change_unc

,future_id,year,cement_emission_fact_coal,cement_emission_fact_diesel,cement_emission_fact_kerosene,cement_emission_fact_natural_gas,cement_frac_biomass,cement_frac_coal,cement_frac_diesel,cement_frac_electric,...,transport_occupancy_rate_train,transport_pkm_aviation,transport_share_diesel_private,transport_share_diesel_truck,transport_tkm_freight_train,transport_tkm_maritime,transport_train13_pkm,transport_train5_pkm,transport_train8_pkm,transport_trip_distance_private
0,0,2015,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
0,0,2016,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
0,0,2017,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
0,0,2018,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
0,0,2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10,2046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.039716,0.938469,1.0,1.0,0.971029,1.0,1.006573,0.975088,1.034949,0.992364
10,10,2047,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.041187,0.936190,1.0,1.0,0.969956,1.0,1.006817,0.974165,1.036244,0.992081
10,10,2048,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.042658,0.933911,1.0,1.0,0.968883,1.0,1.007060,0.973242,1.037538,0.991798
10,10,2049,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.044129,0.931632,1.0,1.0,0.967810,1.0,1.007304,0.972320,1.038832,0.991515


In [150]:
df_strat["variable_name_lower"].unique()

array(['medida_biodigestores', 'medida_captura_c_suelos',
       'medida_uso_eficiente_fertilizante', 'medida_cambio_dieta_bovina',
       'incendio_bosque_nativo', 'incendio_plantaciones_forestales',
       'recuperacion_bn', 'sustitucion_degradacion',
       'tierras_convertidas_a_bn', 'tierras_convertidas_a_plantaciones'],
      dtype=object)

In [102]:
#generate transformed values of uncertainty
array_ed_add_sec_trans = array_ed_add_sec * np.array(df_max_min["max_2050"]) + (1 - array_ed_add_sec) * np.array(df_max_min["min_2050"])
#add in future 0 (0 change = 100%)
array_ed_add_sec_trans = np.concatenate([np.ones((1, array_ed_add_sec_trans.shape[1])), array_ed_add_sec_trans])
#set data frame of baseline percentage changes
df_perc_change_unc = []

In [116]:
i = 30
y = param_years_add_sec[i]
frac_base = vec_ramp_base[i]
frac_unc = vec_ramp_unc[i]
#build weighted array
array_tmp = np.ones(array_ed_add_sec_trans.shape)*(frac_base) + array_ed_add_sec_trans*(frac_unc)



In [117]:
indices_print = indices_fop_all_constant_params + list(np.array(indices_fop_all_constant_params) + 10)
                                                                
                                                                

In [118]:

print(array_tmp[:,[indices_print[x] for x in [0, 10]]])
array_tmp[:,indices_fop_all_constant_params] = array_ed_add_sec_trans[:,indices_fop_all_constant_params].copy()
print("\n\n")
print(array_tmp[:,[indices_print[x] for x in [0, 10]]])


[[1.         1.        ]
 [0.88728259 0.99921402]
 [0.94022647 1.03636472]
 [0.92851765 1.01306655]
 [1.07701698 0.96796884]
 [0.97024097 0.9831024 ]
 [1.05179122 0.95924539]
 [0.85956835 1.02062771]
 [1.10320409 0.98988442]
 [1.13750348 1.00326652]
 [1.01874763 1.02654562]]



[[1.         1.        ]
 [0.86560616 0.99921402]
 [0.92873157 1.03636472]
 [0.91477104 1.01306655]
 [1.09182794 0.96796884]
 [0.96451809 0.9831024 ]
 [1.06175107 0.95924539]
 [0.83256226 1.02062771]
 [1.12305103 0.98988442]
 [1.16394646 1.00326652]
 [1.02235295 1.02654562]]


In [105]:
print(array_tmp[:,indices_fop_all_constant_params].copy())

[[1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [0.88728259 1.12926903 0.91989878 1.02727694 1.09215969 0.92714762
  1.08838857 1.02660782 1.03379264 1.11773715]
 [0.94022647 0.94564495 1.11213675 0.98858665 1.02394966 0.83359914
  1.0237238  1.11867939 0.93304152 1.0223672 ]
 [0.92851765 0.86604043 0.8691899  1.14877501 0.90731502 1.06087314
  1.13711273 0.92989678 1.13781826 0.84237006]
 [1.07701698 0.84121041 1.03133631 0.92478086 0.87134914 1.01047228
  0.93628395 0.84380176 0.85452533 1.15982003]
 [0.97024097 1.14530388 1.13846046 0.85262476 0.97525645 0.9609933
  0.863016   0.87497972 1.12003143 0.87906733]
 [1.05179122 1.06003398 0.97925408 1.1197411  1.16258542 0.88662947
  1.1188907  1.04044646 0.90409734 0.90604176]
 [0.85956835 1.09368273 1.07213715 1.067973   1.04218786 1.13235643
  1.06261881 1.09152645 0.9919565  0.96810999]
 [1.10320409 0.92840581 0.86573132 0.86938907 0.9364284  0.98673932
  0.91963661 1.154922

In [99]:
ast[:,indices_fop_all_constant_params]

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.94341069, 1.04528658, 1.01222264, 0.90992271, 1.09891573,
        0.9337923 , 0.91017297, 0.94103206, 1.01840198, 0.90098622],
       [0.87565111, 1.10053342, 1.09112186, 0.99484972, 1.06234268,
        0.85892276, 0.85186394, 1.11002528, 1.14698492, 0.96475886],
       [1.06368663, 0.86957238, 0.96213298, 1.15365184, 0.91180003,
        1.01158886, 1.16619427, 1.04985569, 0.98391473, 0.97739622],
       [1.06709984, 0.83550075, 1.0656565 , 1.11976524, 0.89645263,
        1.03387512, 0.99272645, 1.14655027, 1.06222608, 1.06738436],
       [0.85747035, 1.11042113, 0.92921475, 1.05577771, 0.94443982,
        1.09347641, 0.93704701, 0.98726258, 0.93287851, 0.85943522],
       [1.13384347, 0.91803782, 0.84880585, 0.84750014, 0.98557335,
        1.10764738, 1.03530547, 1.09743231, 1.10895217, 1.0246225 ],
       [1.02094096, 1.01138411, 0.8659755

In [40]:
indices_fop_all_constant_params

[51, 52, 66, 67, 68, 69, 132, 191, 192, 193]

In [56]:
array_tmp[:,indices_fop_all_constant_params]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [58]:
ed 

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.99301695, 0.99987788,
        1.0021255 ],
       [1.        , 1.        , 1.        , ..., 1.01278871, 1.02530775,
        0.9617324 ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.02199977, 1.01305977,
        1.01328842],
       [1.        , 1.        , 1.        , ..., 1.00249835, 1.00431145,
        1.02848892],
       [1.        , 1.        , 1.        , ..., 0.98779082, 1.01705847,
        0.95499117]])

In [119]:
df_out = pd.read_csv("/Volumes/[C] syme-j-PVM.hidden/Users/jsyme/Documents/Projects/SWCHE076-1000/ade_beta/out/output_multi_sector_analytica.csv")

In [162]:
#read in experimental design
exp_design = pd.read_csv(sr.fp_csv_experimental_design_msec)
exp_design_single_val = pd.read_csv(sr.fp_csv_experimental_design_msec_single_vals)
#setup experimental design columns
exp_design_cols = list(set(exp_design.columns) | set(exp_design_single_val.columns))
exp_design_cols.sort()
#read in parameters and create a dictionary that maps field to sectors
params = pd.read_csv(sr.fp_csv_parameter_ranges)
#pull in master ids to run
df_masters_to_run = pd.read_csv(sr.fp_csv_experimental_design_msec_masters_to_run)
#reduce
exp_design = exp_design[exp_design["master_id"].isin(list(df_masters_to_run["master_id"]))].reset_index(drop = True)

#remove lever delta data
exp_design = exp_design[[x for x in exp_design.columns if not ("lever_delta_" in x)]]
exp_design_single_val = exp_design_single_val[[x for x in exp_design_single_val.columns if not ("lever_delta_" in x)]]
#function to expand single values out
def edsv_expand(n_row):
    #check columns
    cols = [x for x in list(exp_design_single_val.columns) if x not in exp_design.columns]
    n_cols = len(cols)

    if n_cols > 0:
        #expand out
        array_expand = np.array([np.array(exp_design_single_val)[0] for x in range(n_row)])
        #convert to data frae
        df_expand = pd.DataFrame(array_expand, columns = list(exp_design_single_val.columns))
    else:
        df_expand = pd.DataFrame([], columns = None)

    return(df_expand)



#print("Generating wide experimental design (Time elapsed: " + str(np.round((time.time() - t0)/60, 2)) + " minutes)...")
#update experimental design
exp_design = pd.concat([exp_design, edsv_expand(len(exp_design))], axis = 1)
#sort
exp_design = exp_design.sort_values(by = ["master_id", "year"])



In [165]:
exp_design[(exp_design["master_id"] == 1)][["strategy_id", "cumplimiento_metas"]]



,strategy_id,cumplimiento_metas
36,0,0.000000
37,0,0.000000
38,0,0.000000
39,0,0.000000
40,0,0.000000
41,0,0.000000
42,0,0.000000
43,0,0.000000
44,0,0.959029
45,0,0.959029


In [145]:
parameter_table_additional_sectors[parameter_table_additional_sectors["sector"] == "Analytica_waste"]



,sector,time_series_id,strategy_id,type,parameter,parameter_constant_q,min_2050,max_2050,2015,2016,...,2044,2045,2046,2047,2048,2049,2050,variable_name_lower,norm_group_id,lever_group_id
46,Analytica_waste,0,0,incertidumbre,cumplimiento_metas,0,0.8,1.05,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,cumplimiento_metas,42,-1
47,Analytica_waste,0,1,incertidumbre,cumplimiento_metas,0,0.8,1.05,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,cumplimiento_metas,42,-1
48,Analytica_waste,0,0,incertidumbre,ptas_antofa,0,1.0,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ptas_antofa,43,-1
49,Analytica_waste,0,1,incertidumbre,ptas_antofa,0,1.0,1.00,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,ptas_antofa,43,-1
50,Analytica_waste,0,0,incertidumbre,ptas_biobio,0,1.0,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ptas_biobio,44,-1
51,Analytica_waste,0,1,incertidumbre,ptas_biobio,0,1.0,1.00,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,ptas_biobio,44,-1
52,Analytica_waste,0,0,incertidumbre,ptas_valpo,0,1.0,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ptas_valpo,45,-1
53,Analytica_waste,0,1,incertidumbre,ptas_valpo,0,1.0,1.00,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,ptas_valpo,45,-1
54,Analytica_waste,0,0,incertidumbre,ptasls_coq,0,1.0,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ptasls_coq,46,-1
55,Analytica_waste,0,1,incertidumbre,ptasls_coq,0,1.0,1.00,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,ptasls_coq,46,-1


In [156]:
array_tmp[:,indices_fop_all_constant_params]

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.02769992, 1.02251563, 0.91738242, 1.16086577, 0.8533859 ,
        0.96802167, 1.09071598, 0.82877455, 0.97586366, 0.97530079],
       [0.89613581, 1.0772897 , 0.99792914, 0.80110036, 1.06926033,
        1.15323068, 1.07902026, 0.9383535 , 0.94047381, 0.94029889],
       [0.87083407, 1.13244438, 1.05375408, 0.85640667, 0.83081574,
        1.0226799 , 0.95839069, 0.98906463, 1.03294121, 1.19887056],
       [1.1684277 , 0.90536909, 0.85247121, 0.95944258, 1.17653426,
        1.08419693, 0.90791576, 0.86509674, 1.10785056, 1.08069109],
       [0.92825953, 1.11475644, 1.01269237, 1.07368233, 1.13595487,
        0.91410503, 0.85730938, 1.11394323, 1.05631395, 1.00576644],
       [1.12298149, 0.97731027, 0.95161689, 0.99055197, 0.9013616 ,
        0.83990171, 0.82899839, 1.12379194, 1.14003373, 0.87517693],
       [0.96337055, 0.94513002, 1.1933417